In [58]:
import pandas as pd

In [59]:
## Open file
number_samples = 6
bracken_file=pd.read_csv(filepath_or_buffer='/Volumes/PiconCossio/biocomp_tools/MAUS/utils/braken_output_G1_G2.txt', 
                         delimiter='\t', skiprows=8)
bracken_file.head()

,#perc,tot_all,tot_lvl,G1_1_all,G1_1_lvl,G2_1_all,G2_1_lvl,G3_1_all,G3_1_lvl,G4_1_all,G4_1_lvl,G5_1_all,G5_1_lvl,G6_1_all,G6_1_lvl,lvl_type,taxid,name
0,0.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,U,0,unclassified
1,100.1948,1841211,0,314064,0,326812,0,231626,0,341953,0,308615,0,318141,0,R,1,root
2,86.6385,1592096,0,261690,0,233040,0,223694,0,324432,0,265100,0,284140,0,D,3,Bacteria
3,35.1509,645944,0,62650,0,30091,0,114312,0,189540,0,110498,0,138853,0,P,2375,Proteobacteria
4,28.3632,521211,0,29902,0,20580,0,92022,0,169623,0,79585,0,129499,0,C,3303,Gammaproteobacteria


In [60]:
## compute relative abundance (RelAbun)
def compute_relative_abun (df, number_samples=6):
    for i in range(1, number_samples+1):
        i = str(i)
        df[f"G{i}_1_RelAbun"] = df[f"G{i}_1_lvl"]/df[f"G{i}_1_lvl"].sum() * 100
    return df

In [61]:
bracken_file=compute_relative_abun(df=bracken_file, number_samples=number_samples)
min_abundance=0.1
def filter_by_abundance_conditional (df, min_abundance, min_samples):
    ## This is intended to mimic the taxa filter by abundance conditionally of qiime2
    ## Select taxa that have at least n abundance in at least n samples 
    df = df[(df.iloc[:,number_samples*-1:] >= min_abundance).sum(axis=1) >= min_samples]

    return df

bracken_file = filter_by_abundance_conditional (df=bracken_file, 
                                                min_abundance=min_abundance, 
                                                min_samples=2)
bracken_file=compute_relative_abun(df=bracken_file, number_samples=number_samples)
bracken_file

,#perc,tot_all,tot_lvl,G1_1_all,G1_1_lvl,G2_1_all,G2_1_lvl,G3_1_all,G3_1_lvl,G4_1_all,...,G6_1_lvl,lvl_type,taxid,name,G1_1_RelAbun,G2_1_RelAbun,G3_1_RelAbun,G4_1_RelAbun,G5_1_RelAbun,G6_1_RelAbun
7,11.3339,208276,208276,11107,11107,3681,3681,39821,39821,60666,...,50589,G,3723,Pseudomonas,4.617182,1.217947,19.876313,19.584208,15.741497,18.191985
8,1.5269,28058,28058,3392,3392,166,166,189,189,21853,...,641,G,14683,Thiopseudomonas,1.410055,0.054925,0.094338,7.054589,0.674392,0.230506
12,0.3526,6479,6479,95,95,479,479,2532,2532,778,...,1845,G,3711,Acinetobacter,0.039492,0.158489,1.263826,0.251154,0.278368,0.663469
22,0.3680,6762,6762,28,28,547,547,5179,5179,193,...,642,G,50404,Halomonas,0.011640,0.180988,2.585054,0.062304,0.064210,0.230865
105,1.9059,35023,35023,1335,1335,312,312,2177,2177,26830,...,2752,G,46191,Advenella,0.554960,0.103233,1.086631,8.661265,0.600160,0.989629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3249,0.5104,9379,9379,1685,1685,2870,2870,125,125,403,...,662,G,43151,RumEn M2,0.700455,0.949608,0.062393,0.130097,1.348783,0.238058
3250,0.1059,1946,1946,128,128,323,323,51,51,857,...,389,G,43152,uncultured,0.053210,0.106872,0.025456,0.276657,0.073489,0.139886
3251,0.0903,1659,1659,328,328,541,541,53,53,301,...,99,G,43149,Candidatus Methanomethylophilus,0.136350,0.179003,0.026454,0.097169,0.125080,0.035601
3268,0.3592,6601,6601,1464,1464,3188,3188,91,91,41,...,70,G,122,Methanobacterium,0.608585,1.054826,0.045422,0.013236,0.648411,0.025172
